### **Setup**

In [1]:
! pip install --quiet datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
! pip uninstall -y wandb

Found existing installation: wandb 0.19.1
Uninstalling wandb-0.19.1:
  Successfully uninstalled wandb-0.19.1


### **Load Dataset**

In [3]:
from datasets import load_dataset

ner_dataset = load_dataset("rasyosef/amharic-named-entity-recognition", split="train")
ner_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


README.md:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/546k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3465 [00:00<?, ? examples/s]

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 3465
})

In [4]:
ner_dataset.features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-TIME', 'I-TIME', 'B-TTL', 'I-TTL'], id=None), length=-1, id=None)

In [5]:
categories = ner_dataset.features["ner_tags"].feature.names

label2id = {
  label: i for i, label in enumerate(categories)
}

id2label = {
  v: k for k, v in label2id.items()
}

print(id2label)
print(label2id)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-TIME', 8: 'I-TIME', 9: 'B-TTL', 10: 'I-TTL'}
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-TIME': 7, 'I-TIME': 8, 'B-TTL': 9, 'I-TTL': 10}


### **Processing the data**

In [6]:
from transformers import AutoTokenizer

model_id = "rasyosef/roberta-medium-amharic"
tokenizer = AutoTokenizer.from_pretrained(model_id, add_prefix_space=True)

print(tokenizer.tokenize("ከሀገራቸው ከኢትዮጵያ ከወጡ ግማሽ ምዕተ <mask> ተቆጥሯል።"))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

['▁ከ', 'ሀገራቸው', '▁ከኢትዮጵያ', '▁ከወጡ', '▁ግማሽ', '▁ምዕተ', ' <mask>', '▁ተቆጥ', 'ሯል።']


In [7]:
tokenizer.is_fast

True

In [8]:
# Tokenize the pretokenized input by adding is_split_into_words=True
inputs = tokenizer(ner_dataset[0]["tokens"], is_split_into_words=True)
print(inputs.tokens())

['<s>', '▁ኢዴፓ', '▁በየክልሉ', '▁በሚ', 'ንቀሳቀስ', 'በት', '▁ጊዜ', '▁ሁሉ', '▁የሀገሪቱን', '▁አጠቃላይ', '▁ሕግ', 'እንዲሁም', '▁የአካባቢውን', '▁ባህልና', '▁ቋንቋ', '▁አክብሮ', '▁በአካባቢው', '▁የሚገኙ', '▁የፖለቲካ', '▁ድርጅቶችን', 'ም', '▁አክብሮ', 'ና', '▁መብታቸውን', '▁ጠብቆ', '▁በ', 'ጨዋ', 'ነት', '▁ያስተምራል', '▁', '፣', '▁ይማራል', '"', '▁ብለዋል', '▁', '።', '</s>']


In [9]:
print(inputs.word_ids())

[None, 0, 1, 2, 2, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 12, 13, 14, 15, 15, 16, 16, 17, 18, 19, 19, 19, 20, 21, 21, 22, 22, 23, 24, 24, None]


In [10]:
def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word = None
  for word_id in word_ids:
    if word_id != current_word: # start of new word
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)
    elif word_id is None: # special token
      new_labels.append(-100)
    else:
      # same word as pervious token
      label = labels[word_id]
      # If label is B-XXX change it to I-XXX
      if id2label[label].startswith("B-"):
        label = label2id["I-" + id2label[label][2:]]
      new_labels.append(label)
  return new_labels


In [11]:
labels = ner_dataset[2]["ner_tags"]
inputs = tokenizer(ner_dataset[2]["tokens"], is_split_into_words=True)
word_ids = inputs.word_ids()
print(inputs.tokens())
print(labels)
print(align_labels_with_tokens(labels, word_ids))

['<s>', '▁በባህር', '▁ዳር', '▁ዩኒቨርስቲ', 'ና', '▁በጅማ', '▁መምህራን', '▁ኮሌጅ', '▁ለነ', 'ባር', '▁ተማሪዎች', '▁የምግብ', 'ና', '▁የመኝታ', '▁አገልግሎት', '▁ሊያ', 'ቆም', '▁እንደሚችል', '▁ባለፈው', '▁ዓመት', '▁የተሰጠው', '▁ማሳሰቢያ', '▁ከዚህ', '▁ዓመት', '▁ጀምሮ', '▁ተግባራዊ', '▁ይሆናል', '▁በመ', 'ባሉ', '▁ተማሪዎች', '▁ከፍተኛ', '▁ስጋት', 'ና', '▁ጭንቀት', '▁ላይ', '▁', 'መውደቃቸው', 'ን', '▁ተጠቆመ', '▁', '።', '</s>']
[3, 4, 4, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 3, 4, 4, 4, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [12]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(
      examples['tokens'], truncation=True, is_split_into_words=True
  )
  all_labels = examples['ner_tags']
  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels, word_ids))

  tokenized_inputs['labels'] = new_labels
  return tokenized_inputs

In [13]:
tokenized_datasets = ner_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=ner_dataset.column_names,
)
tokenized_datasets

Map:   0%|          | 0/3465 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3465
})

In [14]:
preprocessed_datasets = tokenized_datasets.train_test_split(test_size=0.2, seed=16)
preprocessed_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2772
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 693
    })
})

### **Finetuning**

In [15]:
# Data Collator
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [16]:
batch = data_collator([preprocessed_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    5,    6,    6,    6,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

### **Metrics**

In [17]:
!pip install --quiet seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [18]:
import evaluate

metric = evaluate.load("seqeval")

In [19]:
labels = preprocessed_datasets["train"][3]["labels"]
print(labels)
labels = [categories[i] for i in labels[1:-1]]
print(labels), labels[19]

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


(None, 'O')

In [20]:
predictions = labels.copy()
predictions[19] = "O"
metric.compute(predictions=[predictions], references=[labels])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

In [21]:
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[categories[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [categories[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

### **Defining the Model**

In [22]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_id,
    id2label=id2label,
    label2id=label2id,
)

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/169M [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at rasyosef/roberta-medium-amharic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Number of labels
model.config.num_labels

11

### **Fine-tuning the model**

In [24]:
from transformers import TrainingArguments

args = TrainingArguments(
    "xlm-roberta-base-finetuned-ner",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    lr_scheduler_type="linear",
    num_train_epochs=8,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    seed=32
)

In [25]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=preprocessed_datasets["train"],
    eval_dataset=preprocessed_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

<ipython-input-25-efbf7a4a7c5a>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.244500,0.137123,0.518967,0.721661,0.603756,0.948079
2,0.093200,0.104658,0.678682,0.739618,0.707841,0.964783
3,0.055800,0.120162,0.688458,0.756453,0.720856,0.964443
4,0.033900,0.132492,0.682224,0.771044,0.723920,0.962860
5,0.021200,0.145523,0.712788,0.763187,0.737127,0.965235
6,0.014900,0.164875,0.702321,0.781145,0.739639,0.964971
7,0.009900,0.169237,0.719101,0.790123,0.752941,0.966253
8,0.007700,0.173668,0.708583,0.796857,0.750132,0.965461


TrainOutput(global_step=1392, training_loss=0.060141004279427146, metrics={'train_runtime': 182.8906, 'train_samples_per_second': 121.253, 'train_steps_per_second': 7.611, 'total_flos': 292365305980440.0, 'train_loss': 0.060141004279427146, 'epoch': 8.0})

In [26]:
trainer.evaluate()

{'eval_loss': 0.16923698782920837,
 'eval_precision': 0.7191011235955056,
 'eval_recall': 0.7901234567901234,
 'eval_f1': 0.7529411764705881,
 'eval_accuracy': 0.9662531578748916,
 'eval_runtime': 1.8583,
 'eval_samples_per_second': 372.914,
 'eval_steps_per_second': 23.677,
 'epoch': 8.0}

### **Testing**

In [27]:
from transformers import pipeline

ner_pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

Device set to use cuda:0


In [28]:
ind = 3
text = " ".join(ner_dataset[ind]["tokens"])
print(text)
print(" ".join([str(i) for i in ner_dataset[ind]["ner_tags"]]))
ner_pipe(text)

የሻዕቢያው መሪ ለዚህ ማስፈራሪያቸው እንደ አብነት የተጠቀሙበት ኦጋዴንን ሲሆን የኢሕአዴግ መንግሥት ከቅኝ ግዛት ውሎቹ ላፈንግጥ ቢል ያንን ግዛት ለዘለዓለሙ ሊያጣው እንደሚችል ግልጽ ሊሆንለት ይገባል ሲሉ አስጠንቅቀዋል ።
3 0 0 0 0 0 0 5 0 3 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


[{'entity_group': 'ORG',
  'score': 0.9970722,
  'word': 'የሻዕቢያው',
  'start': 0,
  'end': 6},
 {'entity_group': 'LOC',
  'score': 0.99588746,
  'word': 'ኦጋዴንን',
  'start': 40,
  'end': 45},
 {'entity_group': 'ORG',
  'score': 0.996502,
  'word': 'የኢሕአዴግ መንግሥት',
  'start': 50,
  'end': 62}]

In [29]:
ind = 12
text = " ".join(ner_dataset[ind]["tokens"])
print(text)
print(" ".join([str(i) for i in ner_dataset[ind]["ner_tags"]]))
ner_pipe(text)

በናዝሬት አጠቃላይ ዕድሮች የመሰብሰቢያ አዳራሽ በተካሄደው በዚሁ ስብሰባ ላይ በአዳራሽ ውስጥ በመቀመጥና በመቆም ፣ ከውጭም በመስኮት ቁጥሩ እስከ 1400 የሆነ ሕዝብ መገኘቱን የገለጡት አቶ ልደቱ መታፈስ ሊኖር ይችላል" የሚል ማስፈራሪያ በከተማው ተሠራጭቶ የነበረ ቢሆንም ቁጥራቸው የበዛ ወጣቶች ጐልማሶችና አዛውንቶች በስብሰባው ላይ መካፈላቸውን ፣ ብዙውም በአዳራሽ ጥበት ምክንያትም መመለሱን አመልክተዋል ።
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 9 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


[{'entity_group': 'TTL',
  'score': 0.99870956,
  'word': 'አቶ',
  'start': 117,
  'end': 119},
 {'entity_group': 'PER',
  'score': 0.99788755,
  'word': 'ልደቱ',
  'start': 120,
  'end': 123}]